In [3]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as st
from datetime import timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)



**Görev 1 = Average Rating’i güncel yorumlara göre 
hesaplayınız ve var olan average rating ile 
kıyaslayınız.**


In [4]:
df =  pd.read_csv("../input/amazon-product-reviews/amazon_review.csv")

In [5]:
df["overall"].mean()  ## var olan avarage rating

4.587589013224822

In [6]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'])
analysis_day = df['reviewTime'].max() + timedelta(days=1)

In [7]:
df["days_diff"] = (analysis_day - df['reviewTime']).dt.days 

In [8]:
q25, q50, q75 = df["days_diff"].quantile([0.25, 0.5, 0.75])

In [12]:
average_rating = df.loc[df["days_diff"] <= 25, "overall"].mean() * 28 / 100 + \
df.loc[(df["days_diff"] > q25) & (df["days_diff"] <= q50), "overall"].mean() * 26 / 100 + \
df.loc[(df["days_diff"] > q50) & (df["days_diff"] <= q75), "overall"].mean() * 24 / 100 + \
df.loc[(df["days_diff"] > q75), "overall"].mean() * 22 / 100

## 4.608649946534414


**Görev 2 : Ürün için ürün detay sayfasında 
görüntülenecek 20 review’i belirleyiniz.**

Helpful özelliklerine ve overall puanına göre sıralanması alıcı adına daha faydalı olucaktır

In [22]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score Calculation

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [20]:
df["upvote"] = df["helpful_yes"] 
df["downvote"] = df["total_vote"] - df["helpful_yes"]

In [21]:
df["score_pos_neg_diff"] = df['upvote'] - df['downvote']
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["upvote"], x["downvote"]), axis=1)

In [16]:
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["upvote"], x["downvote"]), axis=1)


In [19]:
shown_comments = df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days_diff,upvote,downvote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,702,1952,68,1884,0.96634,0.95754
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,803,1428,77,1351,0.94884,0.93652
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,579,1568,126,1442,0.92562,0.91214
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1033,422,73,349,0.85253,0.81858
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,158,45,4,41,0.91837,0.80811
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.00000,I own it,1393545600,2014-02-28,283,60,68,283,60,8,52,0.88235,0.78465
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,777,112,27,85,0.80576,0.73214
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.00000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,649,22,3,19,0.88000,0.70044
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.00000,Awesome Card!,1339200000,2012-09-06,823,51,65,823,51,14,37,0.78462,0.67033
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.00000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,807,82,27,55,0.75229,0.66359
